<a href="https://colab.research.google.com/github/wwsalmon/hacklytics-cecil-covid/blob/main/xh_temp_folder/hackalytics_covid_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup

import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import seaborn as sns
import numpy as np
#from matplotlib import font_manager as fm

In [2]:
# Functions to clean the data
clear_plus = lambda x : re.sub('\+','', x)
clear_paren = lambda x : re.sub('\(.*\)', '', x)

take_inf = lambda x : int(re.sub(r'[^0-9]*([0-9]+)[^0-9]*([0-9]*)[^0-9]*', r'\1', x))
take_sup = lambda x : int(x) if x.isdigit() else int(re.sub(r'[0-9]*[^0-9]*([0-9]+)[^0-9]*', r'\1', x))

# Primary

In [42]:
semiclean = pd.read_csv('https://github.com/xh313/hacklytics-cecil-covid/raw/main/xh_temp_folder/covid_primary_ve_studies_semiclean.csv')
col_semiclean = semiclean[['study ID', 'vaccine', 'dose',
                           'VE', 'LCL', 'UCL',
                           'days_since_first_dose', 'population_category',
                           'outcome_category', 'variant_sequenced']] # select columns

# semiclean dataset has eliminated effect of 1+, thus the + could be dropped
col_semiclean['dose'] = col_semiclean.loc[:, 'dose'].apply(clear_plus)

# Remove open intervals
col_semiclean = col_semiclean.dropna(subset=['days_since_first_dose'])
col_semiclean = col_semiclean[~(col_semiclean.days_since_first_dose.str.contains(r'\+|≥|>|<|≤'))]

#col_semiclean['days_since_first_dose'] = col_semiclean.loc[:, 'days_since_first_dose'].apply(clear_plus).apply(clear_paren)

# Duplicating the days_since_first_dose col for interval extraction purposes
col_semiclean.loc[:, 'time_sup'] = col_semiclean['days_since_first_dose']
col_semiclean.loc[:, 'time_inf'] = col_semiclean['days_since_first_dose']

# Interval separation
col_semiclean.loc[:, 'time_sup'] = col_semiclean.loc[:, 'days_since_first_dose'].apply(take_sup)
col_semiclean.loc[:, 'time_inf'] = col_semiclean.loc[:, 'days_since_first_dose'].apply(take_inf)

# Weeks and months to days
week_mask = col_semiclean.days_since_first_dose.str.contains('week')
week_apply = col_semiclean[week_mask]
col_semiclean.loc[week_mask, 'time_sup'] = week_apply['time_sup'] * 7
col_semiclean.loc[week_mask, 'time_inf'] = week_apply['time_inf'] * 7

month_mask = col_semiclean.days_since_first_dose.str.contains('month')
month_apply = col_semiclean[month_mask]
col_semiclean.loc[month_mask, 'time_sup'] = month_apply['time_sup'] * 30
col_semiclean.loc[month_mask, 'time_inf'] = month_apply['time_inf'] * 30

# Take average
clean = col_semiclean # make copy
clean.loc[:, 'days_since_first_dose'] = (clean.time_sup + clean.time_inf) // 2

# Dose datatype convert 
clean.loc[clean.vaccine == 'Janssen', 'dose'] = clean[clean.vaccine == 'Janssen'].dose.replace('final', 1)
clean.loc[clean.vaccine != 'Janssen', 'dose'] = clean[clean.vaccine != 'Janssen'].dose.replace('final', 2)
clean['dose'] = clean['dose'].astype(int)

<ipython-input-42-6c39c084d4ab>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_semiclean['dose'] = col_semiclean.loc[:, 'dose'].apply(clear_plus)


In [43]:
clean#[clean.vaccine == 'Janssen'].dose.unique()

,study ID,vaccine,dose,VE,LCL,UCL,days_since_first_dose,population_category,outcome_category,variant_sequenced,time_sup,time_inf
0,1,Pfizer,1,75.0,72,84.0,28,HCW,infection,NaN,28,28
1,1,Pfizer,1,85.0,71,92.0,28,HCW,symptomatic,NaN,28,28
3,2,Pfizer,2,88.0,84,90.0,7,> 80 years,symptomatic,NaN,7,7
4,3,Pfizer,1,46.0,40,51.0,17,general pop,infection,NaN,21,14
6,3,Pfizer,1,57.0,50,63.0,17,general pop,symptomatic,NaN,21,14
...,...,...,...,...,...,...,...,...,...,...,...,...
3806,365,Moderna,2,55.7,49.1,61.5,17,0-4 years,infection,NaN,35,0
3807,365,Moderna,2,58.6,48.3,66.9,619,0-4 years,infection,NaN,1127,112
3808,365,Moderna,2,59.7,58.3,61.0,17,5-11 years,infection,NaN,35,0
3809,365,Moderna,2,16.0,13.1,18.7,1389,5-11 years,infection,NaN,2527,252


In [44]:
# Export
clean.to_csv('cleaned_ve_data.csv')

In [124]:
col_semiclean['days_since_first_dose'].unique()

array(['28', '>28', '7', '14-21', '>7', '>14', '10', '>0', '>10', '21',
       '35', '>35', '35-48', '14', '>11', '≥15', '22-28', '21-27',
       '35-41', '>21', '28-34', '>12', '>15', '15-21', '≥7', '31',
       '28-34 ', '0', '4 weeks', '84', '28-90', '17', '10-90', '90',
       '14-20', '7-56', '105', '1-7', '42', '56', '14-120', '120', '7-36',
       '157', '127', '168-203', '55-98', '109-159', '28-63', '175', '147',
       '20', '14-27', '112', '<104 days', '>104 days', '70', '14-63',
       '14-69', '140', '70-104', '>120', '98', '15-28', '85-98', '8-28',
       '27', '0 up to 13 days post dose 2', '<90', '14-60', '151-180',
       '121-150', '144', 'day 137', 'day 139', 'day 158', '15-30', '210',
       '1.25 months', '7.25 months', '1 month', '7 months', '2 months',
       '6 months', '252-279', '168-195', '28-55', '140-167', '196-223',
       '7-21', '14-90', '181', '91-180', '15', '14-73', '194-265',
       '56-280', '194-239', '56-249', '134-220', '2-4 weeks', '25 weeks',
  

---
# Booster

In [16]:
col_semiclean.loc[col_semiclean.timing_of_dose_days.str.contains('\+'), :]

ValueError: ignored

In [47]:
semiclean = pd.read_csv('https://github.com/xh313/hacklytics-cecil-covid/raw/main/xh_temp_folder/covid_booster_ve_studies_semiclean.csv')
col_semiclean = semiclean[['Study ID', 'first author', 'booster vaccine', 
                           'reference group',
                           'VE', 'LCL', 'UCL',
                           'timing_of_dose_days', 'population_category',
                           'outcome_category',]] # select columns

# Remove open intervals
col_semiclean = col_semiclean.dropna(subset=['timing_of_dose_days'])
col_semiclean = col_semiclean[~(col_semiclean.timing_of_dose_days.str.contains(r'\+|≥|>|<|≤'))]

# Duplicating the timing_of_dose_days col for interval extraction purposes
col_semiclean.loc[:, 'time_sup'] = col_semiclean['timing_of_dose_days']
col_semiclean.loc[:, 'time_inf'] = col_semiclean['timing_of_dose_days']

# Interval separation
col_semiclean.loc[:, 'time_sup'] = col_semiclean.loc[:, 'timing_of_dose_days'].apply(take_sup)
col_semiclean.loc[:, 'time_inf'] = col_semiclean.loc[:, 'timing_of_dose_days'].apply(take_inf)

# Weeks and months to days
week_mask = col_semiclean.timing_of_dose_days.str.contains('week')
week_apply = col_semiclean[week_mask]
col_semiclean.loc[week_mask, 'time_sup'] = week_apply['time_sup'] * 7
col_semiclean.loc[week_mask, 'time_inf'] = week_apply['time_inf'] * 7

month_mask = col_semiclean.timing_of_dose_days.str.contains('month')
month_apply = col_semiclean[month_mask]
col_semiclean.loc[month_mask, 'time_sup'] = month_apply['time_sup'] * 30
col_semiclean.loc[month_mask, 'time_inf'] = month_apply['time_inf'] * 30

# Take average
clean = col_semiclean # make copy
clean.loc[:, 'timing_of_dose_days'] = (clean.time_sup + clean.time_inf) // 2

In [52]:
clean[clean['Study ID']==110]
#clean.to_csv('cleaned_booster_ve_data_all_ref.csv')

,Study ID,first author,booster vaccine,reference group,VE,LCL,UCL,timing_of_dose_days,population_category,outcome_category,time_sup,time_inf
900,110,Stirrup,Pfizer,complete vaccination with 2 doses at least 84 ...,72.0,67.0,76.0,31,LTCF staff,infection,48,14
902,110,Stirrup,Pfizer,complete vaccination with 2 doses at least 84 ...,63.0,54.0,70.0,31,LTCF staff,infection,48,14
904,110,Stirrup,Pfizer,complete vaccination with 2 doses at least 84 ...,80.0,67.0,88.0,31,LTCF staff,infection,48,14
906,110,Stirrup,Pfizer,complete vaccination with 2 doses at least 84 ...,75.0,52.0,87.0,31,LTCF staff,infection,48,14
1840,110,Stirrup,Moderna,complete vaccination with 2 doses at least 84 ...,72.0,67.0,76.0,31,LTCF staff,infection,48,14
1842,110,Stirrup,Moderna,complete vaccination with 2 doses at least 84 ...,63.0,54.0,70.0,31,LTCF staff,infection,48,14
1844,110,Stirrup,Moderna,complete vaccination with 2 doses at least 84 ...,80.0,67.0,88.0,31,LTCF staff,infection,48,14
1846,110,Stirrup,Moderna,complete vaccination with 2 doses at least 84 ...,75.0,52.0,87.0,31,LTCF staff,infection,48,14


In [41]:
# Sep different groups
unvacc_ref = clean[clean['reference group'].str.contains('unvaccinated')]
unvacc_ref.to_csv('cleaned_booster_ve_data.csv')

In [46]:
col_semiclean.timing_of_dose_days.unique()

array([  17,   46,   21,   49,   15,   20,   45,   57,   37,   24,   10,
         18,   26,  117,   44,   84,   66,   59,   96,   22,  109,  137,
         33,   41,  104,   67,   75,  105,   36,   11,   87,   42,  195,
         30,   12,   14,   90,   63,   19,   31,  139,  225,   52,   74,
        129,  120,   60,   97,   48,  149,  209,   89,   92,  240,   51,
        135,  224,  314,    1,    5,    2, 3267])